In [24]:
import pandas as pd
import numpy as np

# 1. 회원 정보 (ID가 'User_ID')
df_user = pd.DataFrame({
    'User_ID': [101, 102, 103, 104, 105],
    'Name': ['Kevin', 'Tom', 'Alice', 'Mike', 'Sarah'],
    'Region': ['Seoul', 'Busan', 'Seoul', 'Seoul', 'Daegu'],
    'Reg_Date': ['2024-01-01', '2024-01-02', '2024-01-05', '2024-01-07', '2024-01-10']
})

# 2. 주문 내역 (회원ID가 'ID'로 되어 있음 - 불일치!)
df_order = pd.DataFrame({
    'Order_ID': ['ORD-001', 'ORD-002', 'ORD-003', 'ORD-004', 'ORD-005', 'ORD-006'],
    'ID': [101, 102, 101, 104, 101, 105], # 103번 Alice는 주문 안 함 (Ghost)
    'Product': ['Laptop', 'Phone', 'Mouse', 'Laptop', 'Phone', 'Mouse'],
    'Amount': [1500, 800, 50, 1500, 800, 50],
    'Order_Date': ['2024-01-10', '2024-01-11', '2024-01-12', '2024-02-01', '2024-02-05', '2024-02-10']
})

# 3. 월별 매출 목표 (가로로 긴 Wide Format)
df_target = pd.DataFrame({
    'Region': ['Seoul', 'Busan', 'Daegu'],
    'Jan': [5000, 2000, 1000], # 1월 목표
    'Feb': [6000, 2500, 1500]  # 2월 목표
})

print("--- 1. 회원 정보 ---")
print(df_user.head(2))
print("\n--- 2. 주문 내역 ---")
print(df_order.head(2))
print("\n--- 3. 매출 목표 (Wide) ---")
print(df_target)

--- 1. 회원 정보 ---
   User_ID   Name Region    Reg_Date
0      101  Kevin  Seoul  2024-01-01
1      102    Tom  Busan  2024-01-02

--- 2. 주문 내역 ---
  Order_ID   ID Product  Amount  Order_Date
0  ORD-001  101  Laptop    1500  2024-01-10
1  ORD-002  102   Phone     800  2024-01-11

--- 3. 매출 목표 (Wide) ---
  Region   Jan   Feb
0  Seoul  5000  6000
1  Busan  2000  2500
2  Daegu  1000  1500


In [25]:
#1. 데이터 병합 : df_order테이블을 기준으로 df_user을 합쳐라
df_merged = pd.merge(df_order, df_user,
                    how = 'left', #주문내역 모두 살리기
                    left_on='ID', 
                    right_on = 'User_ID', #같은 User_ID의 컬럼명이 달라 지정해줌
                    suffixes =('_Order', '_Reg')) #다른 뜻의 Date컬럼에 이름을 붙여 병합
print("--- 주문 + 회원 정보 ---")
print(df_merged)

--- 주문 + 회원 정보 ---
  Order_ID   ID Product  Amount  Order_Date  User_ID   Name Region    Reg_Date
0  ORD-001  101  Laptop    1500  2024-01-10      101  Kevin  Seoul  2024-01-01
1  ORD-002  102   Phone     800  2024-01-11      102    Tom  Busan  2024-01-02
2  ORD-003  101   Mouse      50  2024-01-12      101  Kevin  Seoul  2024-01-01
3  ORD-004  104  Laptop    1500  2024-02-01      104   Mike  Seoul  2024-01-07
4  ORD-005  101   Phone     800  2024-02-05      101  Kevin  Seoul  2024-01-01
5  ORD-006  105   Mouse      50  2024-02-10      105  Sarah  Daegu  2024-01-10


In [26]:
#2. 구매 안한 유령 회원 찾기
df_ghost = pd.merge(df_user, df_order,
                    left_on='User_ID', 
                    right_on = 'ID',
                    how = 'outer',
                    indicator=True) #가입은 되어있지만 주문이 없음(left_only)인 사람
print("--- 회원 + 주문 정보 ---")
print(df_ghost)

--- 회원 + 주문 정보 ---
   User_ID   Name Region    Reg_Date Order_ID     ID Product  Amount  \
0      101  Kevin  Seoul  2024-01-01  ORD-001  101.0  Laptop  1500.0   
1      101  Kevin  Seoul  2024-01-01  ORD-003  101.0   Mouse    50.0   
2      101  Kevin  Seoul  2024-01-01  ORD-005  101.0   Phone   800.0   
3      102    Tom  Busan  2024-01-02  ORD-002  102.0   Phone   800.0   
4      103  Alice  Seoul  2024-01-05      NaN    NaN     NaN     NaN   
5      104   Mike  Seoul  2024-01-07  ORD-004  104.0  Laptop  1500.0   
6      105  Sarah  Daegu  2024-01-10  ORD-006  105.0   Mouse    50.0   

   Order_Date     _merge  
0  2024-01-10       both  
1  2024-01-12       both  
2  2024-02-05       both  
3  2024-01-11       both  
4         NaN  left_only  
5  2024-02-01       both  
6  2024-02-10       both  


In [27]:
#3. 데이터 구조 변경(Melt)
df_target_long = df_target.melt(
    id_vars='Region',
    var_name='Month',
    value_name='Target_Amount'
    )
print(df_target_long)

  Region Month  Target_Amount
0  Seoul   Jan           5000
1  Busan   Jan           2000
2  Daegu   Jan           1000
3  Seoul   Feb           6000
4  Busan   Feb           2500
5  Daegu   Feb           1500


In [ ]:
#4. 데이터 요약(Pivot)
df_pivot = df_merged.pivot_table(
    index = 'Region',
    columns = 'Product',
    values = 'Amount',
    aggfunc = 'sum')
print(df_pivot)

Product  Laptop  Mouse  Phone
Region                       
Busan       NaN    NaN  800.0
Daegu       NaN   50.0    NaN
Seoul    1500.0   50.0  800.0
